In [1]:
import pandas as pd
from sklearn.datasets import load_wine

wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

In this section we'll check for performance increase when apply two kinds of data transformation:

1. Apply a standard scaler to the data
2. TODO: Add a PCA transformation to the data
3. TODO: Evaluate the clusters. Aske mentioned something with variance.

In [2]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = {
    "scaled": pd.DataFrame(scaler.fit_transform(df), columns=df.columns),
    "raw": df}

data["raw"] = data["raw"].assign(target=wine.target)
data["scaled"] = data["scaled"].assign(target=wine.target)

In [3]:
from pomegranate import GeneralMixtureModel, LogNormalDistribution, GammaDistribution, NormalDistribution, MultivariateGaussianDistribution
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

distributions = {
    "lognormal": LogNormalDistribution,
    "gamma":GammaDistribution,
    "normal":NormalDistribution,
    "multinormal":MultivariateGaussianDistribution
    }

components = range(1,7)

sklearn_models = [KMeans,GaussianMixture]

num_iters = 25

results = {'distribution':[],
           'mutual_information':[],
           'datatype':[],
           'n_components':[]}

for _ in range(3):

    for datatype, df in data.items():

        features, labels = df.drop('target', axis=1, inplace=False), df['target']

        for component in tqdm(components):
            
            for iteration in range(num_iters):
                
                X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.2,stratify=labels)

                for name, distribution in distributions.items():
                    if datatype == 'scaled' and name in ['lognormal','gamma']:
                        score = 0
                    else:
                        # print(f'Running {name} on {datatype} with {component} components')
                        try:
                            model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
                            model.fit(X_train)
                            y_pred = model.predict(X_test)
                            score = adjusted_mutual_info_score(y_test, y_pred)               
                        except ZeroDivisionError:
                            score = 0
                        except ValueError:
                            score = 0
                        except:
                            score = 0

                    results['distribution'].append(name)
                    results['mutual_information'].append(score)
                    results['datatype'].append(datatype)
                    results['n_components'].append(component)

                for model in sklearn_models:

                    #print(f'Running {model.__name__} on {datatype} with {component} components')
                    
                    MODEL = model()
                    if hasattr(MODEL,'n_components'):
                        MODEL.n_components = component
                    if hasattr(MODEL,'n_clusters'):
                        MODEL.n_clusters = component

                    if hasattr(MODEL,'predict'):
                        MODEL.fit(X_train)
                        y_pred = MODEL.predict(X_test)
                    else:
                        y_pred = MODEL.fit_predict(X_test)

                    score = adjusted_mutual_info_score(y_test, y_pred)

                    results['distribution'].append(MODEL.__class__.__name__)
                    results['mutual_information'].append(score)
                    results['datatype'].append(datatype)
                    results['n_components'].append(component)
            
results = pd.DataFrame(results)

 83%|████████▎ | 5/6 [00:19<00:04,  4.93s/it]/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_39446/2366174588.py:45: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_39446/2366174588.py:45: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_39446/2366174588.py:45: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
 67%|██████▋   | 4/6 [00:12<00:07,  3.66s/it]/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_39446/2366174588.py:45: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train

In [10]:
results = results.groupby(['distribution','datatype','n_components']).mean().sort_values('mutual_information',ascending=False)
results.to_csv("results/models.csv",index=True)

In [16]:
results

mutual_information
distribution    datatype n_components                    
GaussianMixture scaled   3                       0.928772
KMeans          scaled   3                       0.902588
GaussianMixture scaled   4                       0.846326
gamma           raw      3                       0.836736
lognormal       raw      4                       0.833122
...                                                   ...
                scaled   2                       0.000000
                         3                       0.000000
                         4                       0.000000
                         5                       0.000000
                raw      1                       0.000000

[72 rows x 1 columns]